In [1]:
cd Desktop/Summer\ research/fem/

/Users/wuyifan/Desktop/Summer research/fem


In [2]:
import numpy as np
import math
import igl
import random
import meshplot as mp
import wildmeshing as wm
import polyfempy as pf
import scipy.sparse as sp

name = "cuboid"
path = "data/"+name+"_l"+".obj"
out = "out/out_"+name+"_260"+".mesh"
fix = 3
force = 1
mag = [random.randint(0,3),random.randint(0,3),random.randint(0,3)]

V, F = igl.read_triangle_mesh(path)
#wm.tetrahedralize(path, out, mute_log=True, edge_length_r=1/10)
solver = pf.Solver()

solver.load_mesh_from_path(out, vismesh_rel_area=1e-3)

v1, f1 = igl.read_triangle_mesh(out)
nv = igl.per_vertex_normals(v1, f1)
print(len(v1))

[2020-06-01 11:33:38.948] [polyfem] [info] Loading mesh...
[2020-06-01 11:33:38.950] [geogram] [info] Loading file out/out_cuboid_260.mesh...
[2020-06-01 11:33:38.958] [geogram] [info] (FP64) nb_v:260 nb_e:0 nb_f:386 nb_b:0 tri:1 dim:3
[2020-06-01 11:33:38.958] [geogram] [info]  nb_tets:872
[2020-06-01 11:33:38.958] [geogram] [info] Attributes on vertices: point[3]
[2020-06-01 11:33:38.978] [polyfem] [info] mesh bb min [-30, -10, -6], max [30, 10, 6]
[2020-06-01 11:33:38.979] [polyfem] [info]  took 0.0313896s
260


In [23]:
"""
minn = np.min(v1, axis=0)
maxx = np.max(v1, axis=0)
def sideset(p):
    #bottom
    if p[1]==minn[1]:
        return 1
    #top
    elif p[1]==maxx[1]:
        return 2
    #left
    elif p[0]==minn[0]:
        return 3
    #right
    elif p[0]==maxx[0]:
        return 4
    #back
    elif p[2]==minn[2]:
        return 5
    #front
    else:
        return 6
solver.set_boundary_side_set_from_bary(sideset)
"""
ps, ts, s = solver.get_boundary_sidesets()
print(len(s))
"""
#left
e1 = ps[s[:,0] == 1, :]
#bottom
e2 = ps[s[:,0] == 2, :]
#right
e3 = ps[s[:,0] == 3, :]
#top
e4 = ps[s[:,0] == 4, :]
#back
e5 = ps[s[:,0] == 5, :]
#front
e6 = ps[s[:,0] == 6, :]

col = np.zeros_like(s); col[s==2] = 2; col[s==3] = 3

#plot2 = mp.plot(V,F, shading={"wireframe": True}, return_plot=True)
plot2 = mp.plot(e1, shading={"point_color": "pink", "point_size": 2}, return_plot=True)
#plot2.add_points(e1, shading={"point_color": "pink", "point_size": 2})
plot2.add_points(e2, shading={"point_color": "red", "point_size": 2})
plot2.add_points(e3, shading={"point_color": "blue", "point_size": 2})
plot2.add_points(e4, shading={"point_color": "purple", "point_size": 2})
plot2.add_points(e5, shading={"point_color": "green", "point_size": 2})
plot2.add_points(e6, shading={"point_color": "black", "point_size": 2})
#plot2.add_points(v1, shading={"point_color": "green", "point_size": 5})

plot2.save("plots/col_cuboid")
"""

1544


'\n#left\ne1 = ps[s[:,0] == 1, :]\n#bottom\ne2 = ps[s[:,0] == 2, :]\n#right\ne3 = ps[s[:,0] == 3, :]\n#top\ne4 = ps[s[:,0] == 4, :]\n#back\ne5 = ps[s[:,0] == 5, :]\n#front\ne6 = ps[s[:,0] == 6, :]\n\ncol = np.zeros_like(s); col[s==2] = 2; col[s==3] = 3\n\n#plot2 = mp.plot(V,F, shading={"wireframe": True}, return_plot=True)\nplot2 = mp.plot(e1, shading={"point_color": "pink", "point_size": 2}, return_plot=True)\n#plot2.add_points(e1, shading={"point_color": "pink", "point_size": 2})\nplot2.add_points(e2, shading={"point_color": "red", "point_size": 2})\nplot2.add_points(e3, shading={"point_color": "blue", "point_size": 2})\nplot2.add_points(e4, shading={"point_color": "purple", "point_size": 2})\nplot2.add_points(e5, shading={"point_color": "green", "point_size": 2})\nplot2.add_points(e6, shading={"point_color": "black", "point_size": 2})\n#plot2.add_points(v1, shading={"point_color": "green", "point_size": 5})\n\nplot2.save("plots/col_cuboid")\n'

In [28]:
settings = pf.Settings()
problem = pf.Problem()
settings.set_pde(pf.PDEs.LinearElasticity)

settings.set_material_params("E", 10000)
settings.set_material_params("nu", 0.35)

# set the displacement value for the sideset id
problem.set_displacement(fix, [0, 0, 0])
problem.set_force(force, mag)
settings.set_problem(problem)
solver.settings(settings)
solver.solve()

p, tri, disp = solver.get_sampled_solution()
m = np.linalg.norm(disp, axis=1)

p_uni, indices, inverse = np.unique(p, return_index=True, return_inverse=True, axis=0)
t_uni = np.array([inverse[tri[:, 0]], inverse[tri[:, 1]], inverse[tri[:, 2]], inverse[tri[:, 3]]]).transpose()
d_uni = disp[indices, :]
m_uni = m[indices]

#print(len(p_uni),len(t_uni))

"""returns the stress tensor on a densly sampled mesh"""
stress = solver.get_stresses()
s_uni = stress[indices, :]

#print(s_uni.shape,len(p_uni))

mises, stress_ten = solver.get_sampled_mises_avg()
mises_u = mises[indices, :]
#print(len(mises))
#print(mises[0])
#print(stress)

n,_ = s_uni.shape
"""
s_vec = np.zeros((n,3))
nor = np.array([[1,0,0],[0,1,0],[0,0,1]])
for i in range(n):
    for j in range(3):
        n_t = nor[j]
        t = n_t.dot(s_uni[i].reshape(3,3))
        t_s = t[0]+t[1]+t[2]
        s_vec[i][j] = t_s
    #print(i, s_vec[i])

s_norm = np.linalg.norm(s_vec, axis=1)
#print(s_norm)
"""      
arr = solver.get_sampled_mises()
arr = arr[indices, :]
#print(len(arr))
#print(len(mises))
""" 
for i in range(len(arr)):
    if not arr[i] == 0:
        print(i, "\nstress: ",mises[i],"\ndisp: ",1000*d_uni[i],"\n")
""" 
#print(len(p), len(tri),len(v1))
print(force,mag)
#print(disp[:10]*100)

[2020-05-30 15:23:02.839] [polyfem] [info] simplex_count: 	872
[2020-05-30 15:23:02.839] [polyfem] [info] regular_count: 	0
[2020-05-30 15:23:02.841] [polyfem] [info] regular_boundary_count: 	0
[2020-05-30 15:23:02.841] [polyfem] [info] simple_singular_count: 	0
[2020-05-30 15:23:02.841] [polyfem] [info] multi_singular_count: 	0
[2020-05-30 15:23:02.841] [polyfem] [info] boundary_count: 	0
[2020-05-30 15:23:02.841] [polyfem] [info] multi_singular_boundary_count: 	0
[2020-05-30 15:23:02.841] [polyfem] [info] non_regular_count: 	0
[2020-05-30 15:23:02.841] [polyfem] [info] non_regular_boundary_count: 	0
[2020-05-30 15:23:02.841] [polyfem] [info] undefined_count: 	0
[2020-05-30 15:23:02.841] [polyfem] [info] total count:	 872
[2020-05-30 15:23:02.841] [polyfem] [info] Building isoparametric basis...
[2020-05-30 15:23:02.850] [polyfem] [info] Computing polygonal basis...
[2020-05-30 15:23:02.850] [polyfem] [info]  took 9.8612e-05s
[2020-05-30 15:23:02.852] [polyfem] [info] hmin: 1.84426
[2

In [32]:
ef = ps[s[:,0] == force, :]
save_path = "plots/"+name+" "+str(fix)+" "+str(force)+str(mag)
v = p+disp
plot = mp.plot(p_uni+d_uni, t_uni, mises_u, return_plot=True)
plot.add_points(ef, shading={"point_color": "white", "point_size": 2})
#plot1 = mp.plot(p_uni+d_uni, t_uni, m_uni*10, return_plot=True)
#plot2 = mp.plot(p_uni+d_uni, t_uni,shading={"wireframe": True}, return_plot=True)
#plot3 = mp.plot(p+disp, tri, m, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.063104…

1

In [58]:
a,_ = s_uni.shape
"""
s_mat = np.zeros((a,3,3))

for i in range(a):
    item = s_uni[i]
    t = item.reshape(3,3)
    s_mat[i] = t

"""
s_vec = np.zeros((a,3))
nor = np.array([[1,0,0],[0,1,0],[0,0,1]])
for i in range(a):
    for j in range(3):
        n_t = nor[j]
        t = n_t.dot(s_uni[i].reshape(3,3))
        t_s = t[0]+t[1]+t[2]
        s_vec[i][j] = t_s
#print(nv.shape, s_vec.shape)

In [193]:
"""
fix = np.zeros((len(p), 1))
for i in range(len(p)):
    pt = p[i]
    if pt[1]>=maxx[1]-0.01:
        fix[i] = 1
    else:
        fix[i] = 0
"""


y = np.hstack((s_vec,d_uni))
print(y[8])

[ 1.04408565e-01  1.41485136e+00 -3.25471386e-01  1.94528775e-03
  9.94701635e-04 -1.17972267e-04]


In [194]:

"""
s_vec = np.zeros((n,3))
nor = np.array([[1,0,0],[0,1,0],[0,0,1]])
for i in range(n):
    for j in range(3):
        n_t = nor[j]
        t = n_t.dot(s_uni[i].reshape(3,3))
        t_s = t[0]+t[1]+t[2]
        s_vec[i][j] = t_s

y = np.hstack((s_vec,d_uni))
"""

'\ndata = np.zeros([len(E)*2])+1\nrow = E[:,0]\ncol = E[:,1]\nrow1 = np.append(row,col)\ncol1 = np.append(col,row)\n\n#print("\n",row1)\n\n\nfor i in range(len(E)):\n    edge = E[i]\n    v1 = edge[0]\n    v2 = edge[1]\n    adj[v1][v2] = 1\n    adj[v1][v2] = 1\n        \n\nadj1 = sp.coo_matrix((data, (row1, col1)))\n#adj2 = sp.coo_matrix((data, (col, row)))\n#print("adj1 = \n",adj1,"\n")\nadj = adj1.toarray()\n#print(adj[0])\n#print(adj[1])\n'